In [1]:
import time
import requests

In [2]:
def fetch(url):
    """
    指定したURLを取得してResponseオブジェクトを返す。一時的なエラーが起きた場合は最大3回リトライする。
    """
    max_retries = 3  # 最大で3回リトライする。
    retries = 0  # 現在のリトライ回数を示す変数。
    while True:
        try:
            print('Retrieving {0}...'.format(url))
            response = requests.get(url)
            print('Status: {0}'.format(response.status_code))
            if response.status_code not in TEMPORARY_ERROR_CODES:
                return response  # 一時的なエラーでなければresponseを返す。

        except requests.exceptions.RequestException as ex:
            # ネットワークレベルのエラー（RequestException）の場合はリトライする。
            print('Exception occured: {0}'.format(ex))

        retries += 1
        if retries >= max_retries:
            raise Exception('Too many retries.')  # リトライ回数の上限を超えた場合は例外を発生させる。

        wait = 2**(retries - 1)  # 指数関数的なリトライ間隔を求める（**はべき乗を表す演算子）。
        print('Waiting {0} seconds...'.format(wait))
        time.sleep(wait)  # ウェイトを取る。

In [3]:
TEMPORARY_ERROR_CODES = (408, 500, 502, 503, 504)  # 一時的なエラーを表すステータスコード。

response = fetch('http://httpbin.org/status/200,404,503')
if 200 <= response.status_code < 300:
  print('Success!')
else:
    print('Error!')

Retrieving http://httpbin.org/status/200,404,503...
Status: 200
Success!


In [4]:
!pip install retrying

In [5]:
import requests
from retrying import retry  # pip install retrying

TEMPORARY_ERROR_CODES = (408, 500, 502, 503, 504)  # 一時的なエラーを表すステータスコード。


def main():
    """
    メインとなる処理。
    """
    response = fetch('http://httpbin.org/status/200,404,503')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')

# stop_max_attempt_numberは最大リトライ回数を指定する。
# wait_exponential_multiplierは指数関数的なウェイトを取る場合の、初回のウェイトをミリ秒単位で指定する。
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000)
def fetch(url):
    """
    指定したURLを取得してResponseオブジェクトを返す。一時的なエラーが起きた場合は最大3回リトライする。
    """
    print('Retrieving {0}...'.format(url))
    response = requests.get(url)
    print('Status: {0}'.format(response.status_code))
    if response.status_code not in TEMPORARY_ERROR_CODES:
        return response  # 一時的なエラーでなければresponseを返す。

    # 一時的なエラーの場合は例外を発生させてリトライする。
    raise Exception('Temporary Error: {0}'.format(response.status_code))

if __name__ == '__main__':
    main()

Retrieving http://httpbin.org/status/200,404,503...
Status: 503
Retrieving http://httpbin.org/status/200,404,503...
Status: 200
Success!


In [6]:
!pip install CacheControl

In [7]:
import requests
from cachecontrol import CacheControl  # pip install CacheControl


session = requests.session()
cached_session = CacheControl(session)  # sessionをラップしたcached_sessionを作る。

# 1回目はキャッシュがないので、サーバーから取得しキャッシュする。
response = cached_session.get('https://docs.python.org/3/')
print(response.from_cache)  # False

# 2回目はETagとLast-Modifiedの値を使って更新されているかを確認する。
# 更新されていない場合のコンテンツはキャッシュから取得するので高速に処理できる。
response = cached_session.get('https://docs.python.org/3/')
print(response.from_cache)  # True

False
True


In [8]:
import re

value = '3,000'
if not re.search(r'^[0-9,]+$', value):  # 数字とカンマのみを含む正規表現にマッチするかチェックする。
    raise ValueError('Invalid price')  # 正しい値でない場合は例外を発生させる。

In [9]:
!pip install voluptuous

In [10]:
from voluptuous import Schema, Match
# 次の4つのルールを持つスキーマを定義する。
schema = Schema({  # ルール1：オブジェクトはdictである。
    'name': str,   # ルール2：nameの値は文字列である。
    'price': Match(r'^[0-9,]+$'),  # ルール3：priceの値は正規表現にマッチする。
}, required=True)  # ルール4：dictのキーは必須である。

# Schemaオブジェクトを関数として呼び出すと、引数のオブジェクトを対象にバリデーションを行う。
schema({
    'name': 'ぶどう',
    'price': '3,000',
})  # スキーマに適合するので例外は発生しない。

schema({
    'name': None,
    'price': '3,000',
})  # スキーマに適合しないので、例外MultipleInvalidが発生する。

MultipleInvalid: expected str for dictionary value @ data['name']